# L3: Multi-agent Customer Support Automation (with Hugging Face)

This notebook runs the same customer support automation system, but has been modified to use a free, powerful LLM from Hugging Face instead of OpenAI.

### Your To-Do:
1. In Google Colab, click on the key icon on the left sidebar (`Secrets`).
2. Create a new secret named `HF_TOKEN`.
3. Paste your Hugging Face API token into the value field.
4. Enable the toggle to make this secret available to the notebook.
5. Run the cells below!

First, we need to install the necessary libraries. We've added `langchain-huggingface` to connect to our new model.

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain-huggingface

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

### Import Libraries and Set Up the Free LLM

In [ ]:
import os
from google.colab import userdata
from crewai import Agent, Task, Crew
from langchain_huggingface import HuggingFaceHub

In [ ]:
# Set up the Hugging Face API token from Colab Secrets
os.environ["HUGGINGFACE_API_TOKEN"] = userdata.get('HF_TOKEN')

# Instantiate the free LLM from Hugging Face
# Mixtral-8x7B is a powerful and popular open-source model
huggingface_llm = HuggingFaceHub(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", 
    model_kwargs={"temperature": 0.7, "max_new_tokens": 1500}
)

## Role Playing, Focus and Cooperation

Now we define our agents. Notice the new `llm=huggingface_llm` parameter. This is where we tell each agent to use our free model.

In [ ]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True,
    llm=huggingface_llm
)

In [ ]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
    llm=huggingface_llm
)

## Tools, Guardrails and Memory

The rest of the workflow is exactly the same. The agents will use the tools and follow the tasks as defined, but all their reasoning will be powered by the Hugging Face model.

In [ ]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [ ]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

### Creating Tasks

In [ ]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [ ]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

In [ ]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

### Running the Crew

**Note**: LLMs can provide different outputs for the same input. Because we are using a different model, the output you get will likely be different from the original tutorial, but the process the agents follow will be the same.

In [ ]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

- Display the final result as Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result)